## RAG W/ Gemini free API key
Author : Saddik Imad
<br/>
Date : 11/01/2024
<br/>
Copyright 2023 Google LLC.
<br/>

For more details, visit google's [tutorial](https://ai.google.dev/examples/doc_search_emb)

**Table of contents**<a id='toc0_'></a>    
  - [Importing libraries](#toc1_1_1_)    
  - [The Gemini API](#toc1_1_2_)    
    - [The API Key](#toc1_1_2_1_)    
    - [Available embedding models](#toc1_1_2_2_)    
  - [Topic specific dataset](#toc1_1_3_)    
    - [JSON format](#toc1_1_3_1_)    
    - [PDF format](#toc1_1_3_2_)    
  - [The embedding database](#toc1_1_4_)    
    - [Changes to the new embedding models](#toc1_1_4_1_)    
    - [Vector database](#toc1_1_4_2_)    
  - [Getting the relevant documents](#toc1_1_5_)    
  - [Prompting the Gemini model](#toc1_1_6_)    
  - [Generating the response](#toc1_1_7_)    
    - [Getting the model](#toc1_1_7_1_)    
    - [Prompting the model](#toc1_1_7_2_)    
  - [The pipeline](#toc1_1_8_)    

### <a id='toc1_1_1_'></a>[Importing libraries](#toc0_)

In [ ]:
# !pip install google-generativeai==0.3.2
# !pip install chromadb
# !pip install pandas
# !pip install PyPDF2
# !pip install python-dotenv

In [1]:
import os
from dotenv import load_dotenv
from pprint import pprint

import pandas as pd

import chromadb
from chromadb import Documents, EmbeddingFunction, Embeddings

import google.generativeai as genai

from IPython.display import Markdown

/home/imad/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In this tutorial the version of `google.generativeai` library was **0.3.2**

In [2]:
genai.__version__

'0.3.2'

### <a id='toc1_1_2_'></a>[The Gemini API](#toc0_)

#### <a id='toc1_1_2_1_'></a>[The API Key](#toc0_)

If you don't have an API Key, create one [here](https://makersuite.google.com/app/apikey).

In [12]:
load_dotenv()

api_key = os.getenv('GEMINI_API_KEY')
genai.configure(api_key=api_key)

#### <a id='toc1_1_2_2_'></a>[Available embedding models](#toc0_)

In [13]:
for m in genai.list_models():
    if 'embedContent' in m.supported_generation_methods:
        print(m.name)

models/embedding-001


### <a id='toc1_1_3_'></a>[Topic specific dataset](#toc0_)

#### <a id='toc1_1_3_1_'></a>[JSON format](#toc0_)

In [14]:
import json

with open('../data/data.json') as f:
    data = json.load(f)

In this example, the JSON file was formatted as follows :

```python
{
    "instruction": "...",
    "input": "...",
    "output": "..."
}
```

In [15]:
pprint(data[0])

{'input': 'Definition de l’Environnement',
 'instruction': '',
 'output': 'Selon la norme ISO 14001, l’environnement est un milieu dans '
           'lequel un organisme fonctionne, incluant l’air, l’eau, la terre, '
           'les ressources naturelles, la flore, la faune, les êtres humains '
           'et leurs interrelations.'}


We try to take each block and convert it into a single string which concatenates the 3 values for the 3 keys.

In [16]:
documents = []

for item in data:
    entry = ""
    if item['instruction'] != '':
        entry += f"Instruction : {item['instruction']}\n"

    if item['input'] != '':
        entry += f"Input : {item['input']}\n"

    if item['output'] != '':
        entry += f"Output : {item['output']}"

    documents.append(entry)

len(documents)

398

In [20]:
pprint(documents[0])

('Input : Definition de l’Environnement\n'
 'Output : Selon la norme ISO 14001, l’environnement est un milieu dans lequel '
 'un organisme fonctionne, incluant l’air, l’eau, la terre, les ressources '
 'naturelles, la flore, la faune, les êtres humains et leurs interrelations.')


#### <a id='toc1_1_3_2_'></a>[PDF format](#toc0_)

In [21]:
from PyPDF2 import PdfReader

Let's satrt by reading the PDF file and extract the text from it.

In [22]:
def extract_text_from_pdf(file_path):
    pdf_reader = PdfReader(file_path)
    num_pages = len(pdf_reader.pages)

    page_offset = 7
    text = ""

    for page in range(page_offset, num_pages):
        text += pdf_reader.pages[page].extract_text()

    return text


text = extract_text_from_pdf('../books/iso_14001.pdf')
print(text)

a:: 
0 z u... 
<l'. 
LI) 
ri 
0 
N 
© ..._, 
;:;. 
01 ·;: 
>­
Cl.. 
0 u Au cœur de /'ISO 14001:2015 
4 Le retour d'expériences de la certificat ion ISO 14001:2004......... 103 
4.1 Un outil qui a fait ses preuves: le référentiel ISO 14001:2004 ...... 105 
4.2 Retour sur les motivations des entreprises .................................... 110 
4.3 La certification du système de management environnemental. ..... 114 
VI Partie Il 
ISO 14001:2015, une évolution s'appuyant 
sur les attentes des utilisateurs 
5 Le domaine d'application : vers une perspective 
de cycle de vie................................................................................... 121 
6 Le modèle PDCA: une approche systémique............................... 123 
6.1 Une nouvelle structure des référentiels de système 
de management . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 124 
6.2 Un enrichissement du lex

In the previous output, we can see that we have some work to do, the text contains a lot of characters that should be removed. The following function helps remove these unwanted characters but in your case you might want to spend more time cleaning the text.

In [23]:
def clean_extracted_text(text):
    cleaned_text = ""

    for i, line in enumerate(text.split('\n')):
        if len(line) > 10 and i > 70:
            cleaned_text += line + '\n'

    cleaned_text = cleaned_text.replace('.', '')
    cleaned_text = cleaned_text.replace('~', '')
    cleaned_text = cleaned_text.replace('©', '')
    cleaned_text = cleaned_text.replace('_', '')
    cleaned_text = cleaned_text.replace(';:;', '')
    return cleaned_text

In [24]:
cleaned_text = clean_extracted_text(text)
len(cleaned_text)

800456

Now, let's use `RecursiveCharacterTextSplitter` to split the cleaned text into chunks so that we can store them in the vector database.

In [25]:
from langchain.text_splitter import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    length_function=len,
    add_start_index=True,
)

Let's print the first chunk and see how it looks.

In [26]:
texts = text_splitter.create_documents([cleaned_text])
pprint(texts[0].page_content)

('r: \n'
 '0 u Remerciements \n'
 'Nous adressons nos vifs remerciements à Hervé Ross-Carré , responsable \n'
 'développement environnement et économie circulaire du Groupe AFNOR, \n'
 'dont les conversations enrichissantes ont contribué au contenu de cet \n'
 'Nous remercions également Sophie Cluse!, responsable développement et \n'
 'management du risque du Groupe AFNOR, pour ses apports sur la notion \n'
 "« d'approche par les risques » \n"
 'Enfin, nous tenons à remercier vivement les nombreuses personnes \n'
 "rencontrées au cours de nos missions (directeurs d'entreprise , "
 'responsables \n'
 'environnement , responsables QSE, sans oublier tous leurs collaborateurs) \n'
 'avec qui, depuis près de vingt ans, nous progressons pour la mise en place \n'
 'pragmatique du management environnemental dans chacune de leurs \n'
 'entreprises \n'
 ' r: \n'
 '  \n'
 '0 u Tableau comparatif \n'
 'des deux versions \n'
 'de la norme ISO 14001 \n'
 'ISO 14001 :2015 ISO 14001:2004 \n'
 "Titre d

Now, we need to create the documents list in order for us to be able to pass it to the `create_chroma_db` method.

In [29]:
documents = []

for chunk in texts:
    documents.append(chunk.page_content)

pprint(documents[0])

('r: \n'
 '0 u Remerciements \n'
 'Nous adressons nos vifs remerciements à Hervé Ross-Carré , responsable \n'
 'développement environnement et économie circulaire du Groupe AFNOR, \n'
 'dont les conversations enrichissantes ont contribué au contenu de cet \n'
 'Nous remercions également Sophie Cluse!, responsable développement et \n'
 'management du risque du Groupe AFNOR, pour ses apports sur la notion \n'
 "« d'approche par les risques » \n"
 'Enfin, nous tenons à remercier vivement les nombreuses personnes \n'
 "rencontrées au cours de nos missions (directeurs d'entreprise , "
 'responsables \n'
 'environnement , responsables QSE, sans oublier tous leurs collaborateurs) \n'
 'avec qui, depuis près de vingt ans, nous progressons pour la mise en place \n'
 'pragmatique du management environnemental dans chacune de leurs \n'
 'entreprises \n'
 ' r: \n'
 '  \n'
 '0 u Tableau comparatif \n'
 'des deux versions \n'
 'de la norme ISO 14001 \n'
 'ISO 14001 :2015 ISO 14001:2004 \n'
 "Titre d

### <a id='toc1_1_4_'></a>[The embedding database](#toc0_)

We will create a [custom function](https://docs.trychroma.com/embeddings#custom-embedding-functions) for performing embedding using the Gemini API. By inputting a set of documents into this custom function, we will receive vectors, or embeddings of the documents.

#### <a id='toc1_1_4_1_'></a>[Changes to the new embedding models](#toc0_)

For the new embeddings model, embedding-001, there is a new task type parameter and the optional title (only valid with task_type=`RETRIEVAL_DOCUMENT`).

These new parameters apply only to the newest embeddings models.The task types are:

Task Type | Description
---       | ---
RETRIEVAL_QUERY	| Specifies the given text is a query in a search/retrieval setting.
RETRIEVAL_DOCUMENT | Specifies the given text is a document in a search/retrieval setting.
SEMANTIC_SIMILARITY	| Specifies the given text will be used for Semantic Textual Similarity (STS).
CLASSIFICATION	| Specifies that the embeddings will be used for classification.
CLUSTERING	| Specifies that the embeddings will be used for clustering.

In [30]:
class GeminiEmbeddingFunction(EmbeddingFunction):
    def __call__(self, input: Documents) -> Embeddings:
        model = 'models/embedding-001'
        # for better results, try to provide a title for each input if the corpus is covering a lot of domains
        title = "Systeme de management de l'environnement"

        return genai.embed_content(
            model=model,
            content=input,
            task_type="retrieval_document",
            title=title)["embedding"]

#### <a id='toc1_1_4_2_'></a>[Vector database](#toc0_)

The `create_chroma_db` function will try to create a new database if it doesn't exists or use the existing one in the path that you specify, in this example the path is `"../database/"`. Then we will loop over the documents and append them with their respective embeddings to the database.

We used `time.sleep()` because the free API has a rate limit of 60 requests per minute.

In [33]:
import time
from tqdm import tqdm

In [37]:
def create_chroma_db(documents, name):
    chroma_client = chromadb.PersistentClient(path="../database/")

    db = chroma_client.get_or_create_collection(
        name=name, embedding_function=GeminiEmbeddingFunction())

    initiali_size = db.count()
    for i, d in tqdm(enumerate(documents), total=len(documents), desc="Creating Chroma DB"):
        db.add(
            documents=d,
            ids=str(i + initiali_size)
        )
        time.sleep(0.5)
    return db


def get_chroma_db(name):
    chroma_client = chromadb.PersistentClient(path="../database/")
    return chroma_client.get_collection(name=name, embedding_function=GeminiEmbeddingFunction())

In [38]:
db = create_chroma_db(documents, "sme_db")
db.count()

1301

Let's see if the database contains anything

In [39]:
pd.DataFrame(db.peek(5))

,ids,embeddings,metadatas,documents,uris,data
0,0,"[0.03734161704778671, -0.056118182837963104, -...",None,Definition de l’Environnement - Selon la norme...,None,None
1,1,"[0.02191094495356083, -0.058092981576919556, -...",None,C'est quoi le management environnemental ? - L...,None,None
2,10,"[0.03508787229657173, -0.06999852508306503, -0...",None,Qu’est ce que l’ISO 14001 ? - L'ISO 14001 est ...,None,None
3,100,"[0.009700464084744453, -0.0605049803853035, -0...",None,Qu'est-ce que la gestion des risques selon l'I...,None,None
4,1000,"[0.026140427216887474, -0.06409834325313568, -...",None,dégraissage \nContrôles et autocontrôles \nArt...,None,None


The document is embedded into a vector with 768 dimensions

In [40]:
len(pd.DataFrame(db.peek(5)).iloc[0]["embeddings"])

768

### <a id='toc1_1_5_'></a>[Getting the relevant documents](#toc0_)

Chroma collections can be queried in a variety of ways, using the `.query` method. we can query by a set of `query_texts`, Chroma will first embed each `query_text` with the collection's embedding function defined above, and then perform the query with the generated embedding.

In [41]:
def get_relevant_passages(query, db, n_results=5):
    passages = db.query(query_texts=[query], n_results=n_results)[
        'documents'][0]
    return passages

In [42]:
question = "Quels sont les outils à utiliser dans la première étape du Contexte de l’organisme ?"
passages = get_relevant_passages(question, db, n_results=5)

Markdown(passages[0])

Explique le Système de management environnemental. - La quatrième étape du "Contexte de l'organisme" concerne le système de management environnemental. Elle implique l'identification des processus nécessaires au fonctionnement de ce système, une exigence introduite dans la version 2015 de la norme ISO 14001. Il est essentiel de définir les processus, examiner les interactions entre eux, et, à l'intérieur de chaque processus, déterminer comment intégrer les aspects environnementaux. Pour soutenir cette démarche, deux outils sont souvent utilisés : la cartographie des processus, qui permet de visualiser les processus et leurs interactions, et la matrice d'interaction des processus, qui offre une vue détaillée des relations entre les différents processus.

### <a id='toc1_1_6_'></a>[Prompting the Gemini model](#toc0_)

Now that we have found the relevant passages in our set of documents, we can use them to construct a prompt to pass into the Gemini API.

In [79]:
def make_prompt(query, relevant_passage):
    escaped = relevant_passage.replace("'", "").replace('"', "")
    # prompt = f"""question : {query}.\n
    # Votre réponse :
    # """

    prompt = f"""question : {query}.\n
    Informations supplémentaires:\n {escaped}\n
    Si vous trouvez que la question n'a aucun rapport avec les informations supplémentaires, vous pouvez l'ignorer et répond par 'OUT OF CONTEXT'.\n
    Votre réponse :
    """

    # prompt = f"""question : {query}.\n
    # Informations supplémentaires:\n {escaped}\n
    # Si vous trouvez que la question n'a aucun rapport avec les informations supplémentaires, vous pouvez l'ignorer et répond par 'OUT OF CONTEXT' si la question est hors contexte en premier lieu et après répond à la question même si elle est hors context en clarifiant au utilisateur que cette réponse n'a aucune relation avec le context.\n
    # Votre réponse :
    # """

    # prompt = f"""Les questions qui vont être posé ont une relation avec le système de management de l'environnement. Voilà la question : {query}.\nEssayer de répondre à la question en utilisant les informations supplémentaires suivantes qui peuvent t'aider à répondre à la question.\nLes informations supplémentaires:\n {escaped}
    # Votre réponse :
    # """

    return prompt

We will take the relevant documents that we got by using the `.query` method and convert them from a list into a string. This string represents the context that will given to the model along side the question in order to get good results.

In [58]:
def convert_pasages_to_list(passages):
    context = ""

    for passage in passages:
        context += passage + "\n"

    return context

In [59]:
prompt = make_prompt(question, convert_pasages_to_list(passages))
Markdown(prompt)

question : Donne-moi le nombre de planetes dans le systeme solaire.

    Votre réponse :
    

### <a id='toc1_1_7_'></a>[Generating the response](#toc0_)

#### <a id='toc1_1_7_1_'></a>[Getting the model](#toc0_)

In [47]:
model = genai.GenerativeModel('gemini-pro')

#### <a id='toc1_1_7_2_'></a>[Prompting the model](#toc0_)

In [48]:
answer = model.generate_content(prompt)
Markdown(answer.text)

Matrice SWOT et Analyse PESTEL

### <a id='toc1_1_8_'></a>[The pipeline](#toc0_)

Now, we will combine everything to create the following pipeline :
1. Provide the question.
2. Search the Chroma database for relevant documents (passages).
3. Convert the passages from a list to a string (context).
4. Create the prompt.
5. Give the question + context to the model.
6. Get the answer.

In [80]:
# Step 1
# question = "Donne-moi le nombre de planetes dans le systeme solaire"
question = "Quelles sont les nouveautés de la norme ISO 14001 version 2015 ?"

# Step 2
db = get_chroma_db("sme_db")
passages = get_relevant_passages(question, db, n_results=5)

# Step 3
context = convert_pasages_to_list(passages)

# Step 4
prompt = make_prompt(question, context)

# Step 5
model = genai.GenerativeModel('gemini-pro')
answer = model.generate_content(prompt)

# Step 6
Markdown(answer.text)

L’adoption d’une perspective de cycle de vie, pour que les aspects environnementaux soient abordés de la conception jusqu’à la fin de vie.